In [ ]:
!pip install faker==18.11.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.7 MB/s eta 0:00:00
  Attempting uninstall: faker
    Found existing installation: Faker 37.4.2
    Uninstalling Faker-37.4.2:
      Successfully uninstalled Faker-37.4.2


In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Script original com ajustes
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from faker import Faker
import calendar

# Configurar seed para reprodutibilidade
np.random.seed(42)
random.seed(42)
fake = Faker('pt_BR')
Faker.seed(42)

print("Gerando dados sintéticos para e-commerce...")

# =============================================================================
# 1. GERAR TABELA DE CLIENTES (10.000 clientes)
# =============================================================================
print("Gerando clientes...")

customers_data = []
for i in range(1, 10001):
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2024, 12, 31)
    registration_date = fake.date_between(start_date=start_date, end_date=end_date)

    customers_data.append({
        'customer_id': i,
        'customer_name': fake.name(),
        'email': fake.email(),
        'registration_date': registration_date,
        'city': fake.city(),
        'state': fake.state_abbr()
    })

customers_df = pd.DataFrame(customers_data)

# =============================================================================
# 2. GERAR TABELA DE PRODUTOS (500 produtos)
# =============================================================================
print("Gerando produtos...")

categories = ['Eletrônicos', 'Casa & Jardim', 'Esportes', 'Moda', 'Livros']
brands = ['Samsung', 'Apple', 'Sony', 'LG', 'Philips', 'Nike', 'Adidas', 'Zara',
          'H&M', 'Generic', 'Premium', 'Basic', 'Pro', 'Elite', 'Standard']

products_data = []
for i in range(1, 501):
    category = random.choice(categories)
    brand = random.choice(brands)

    if category == 'Eletrônicos':
        price = round(np.random.uniform(100, 3000), 2)
    elif category == 'Casa & Jardim':
        price = round(np.random.uniform(20, 800), 2)
    elif category == 'Esportes':
        price = round(np.random.uniform(30, 500), 2)
    elif category == 'Moda':
        price = round(np.random.uniform(25, 300), 2)
    else:  # Livros
        price = round(np.random.uniform(15, 120), 2)

    product_names = {
        'Eletrônicos': ['Smartphone', 'Laptop', 'Tablet', 'Fone de Ouvido', 'TV', 'Câmera'],
        'Casa & Jardim': ['Sofá', 'Mesa', 'Luminária', 'Vaso', 'Tapete', 'Espelho'],
        'Esportes': ['Tênis', 'Camisa', 'Bola', 'Raquete', 'Bicicleta', 'Mochila'],
        'Moda': ['Camiseta', 'Calça', 'Vestido', 'Sapato', 'Bolsa', 'Óculos'],
        'Livros': ['Romance', 'Ficção', 'Técnico', 'Biografia', 'Infantil', 'Autoajuda']
    }

    base_name = random.choice(product_names[category])
    product_name = f"{brand} {base_name} {fake.word().title()}"

    products_data.append({
        'product_id': i,
        'product_name': product_name,
        'category': category,
        'price': price,
        'brand': brand
    })

products_df = pd.DataFrame(products_data)

# =============================================================================
# 3. GERAR TABELA DE PEDIDOS (50.000 pedidos)
# =============================================================================
print("Gerando pedidos...")

orders_data = []
order_items_data = []

def get_seasonal_weight(month):
    weights = {1: 0.8, 2: 0.7, 3: 0.8, 4: 0.9, 5: 1.0, 6: 0.9,
               7: 0.9, 8: 0.8, 9: 0.9, 10: 1.1, 11: 1.4, 12: 1.3}
    return weights.get(month, 1.0)

for i in range(1, 50001):
    weights = [get_seasonal_weight(m) for m in range(1, 13)]
    total_weight = sum(weights)
    month = np.random.choice(range(1, 13), p=[w/total_weight for w in weights])

    _, last_day = calendar.monthrange(2024, month)
    day = random.randint(1, last_day)
    order_date = datetime(2024, month, day)

    customer_id = random.randint(1, 10000)

    status = np.random.choice(['Entregue', 'Processando', 'Cancelado'],
                             p=[0.85, 0.10, 0.05])

    num_items = np.random.choice([1, 2, 3, 4, 5], p=[0.4, 0.3, 0.2, 0.08, 0.02])

    selected_products = random.sample(range(1, 501), num_items)
    total_amount = 0

    for j, product_id in enumerate(selected_products):
        quantity = np.random.choice([1, 2, 3], p=[0.7, 0.25, 0.05])

        base_price = products_df[products_df['product_id'] == product_id]['price'].iloc[0]
        unit_price = round(base_price * np.random.uniform(0.8, 1.05), 2)

        item_total = quantity * unit_price
        total_amount += item_total

        order_items_data.append({
            'order_item_id': len(order_items_data) + 1,
            'order_id': i,
            'product_id': product_id,
            'quantity': quantity,
            'unit_price': unit_price
        })

    orders_data.append({
        'order_id': i,
        'customer_id': customer_id,
        'order_date': order_date,
        'total_amount': round(total_amount, 2),
        'status': status
    })

orders_df = pd.DataFrame(orders_data)
order_items_df = pd.DataFrame(order_items_data)

# =============================================================================
# 4. SALVAR DADOS EM CSV
# =============================================================================
print("Salvando dados em CSV...")

# Criar pasta no Google Drive, se não existir
import os
os.makedirs('/content/drive/My Drive/Ecommerce_Dataset', exist_ok=True)

customers_df.to_csv('/content/drive/My Drive/Ecommerce_Dataset/customers.csv', index=False)
products_df.to_csv('/content/drive/My Drive/Ecommerce_Dataset/products.csv', index=False)
orders_df.to_csv('/content/drive/My Drive/Ecommerce_Dataset/orders.csv', index=False)
order_items_df.to_csv('/content/drive/My Drive/Ecommerce_Dataset/order_items.csv', index=False)

# =============================================================================
# 5. CRIAR SCRIPTS SQL PARA CRIAÇÃO DAS TABELAS
# =============================================================================
print("Criando scripts SQL...")

sql_create_tables = """
-- Criar banco de dados
CREATE DATABASE IF NOT EXISTS ecommerce_test;
USE ecommerce_test;

-- Tabela de clientes
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(100),
    email VARCHAR(100),
    registration_date DATE,
    city VARCHAR(50),
    state VARCHAR(50)
);

-- Tabela de produtos
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(200),
    category VARCHAR(50),
    price DECIMAL(10,2),
    brand VARCHAR(50)
);

-- Tabela de pedidos
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL(10,2),
    status VARCHAR(20),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

-- Tabela de itens do pedido
CREATE TABLE order_items (
    order_item_id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT,
    unit_price DECIMAL(10,2),
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
"""

with open('/content/drive/My Drive/Ecommerce_Dataset/create_tables.sql', 'w', encoding='utf-8') as f:
    f.write(sql_create_tables)

# =============================================================================
# 6. EXIBIR ESTATÍSTICAS DOS DADOS GERADOS
# =============================================================================
print("\n" + "="*60)
print("DADOS GERADOS COM SUCESSO!")
print("="*60)

print(f"\n📊 ESTATÍSTICAS DOS DADOS:")
print(f"• Clientes: {len(customers_df):,}")
print(f"• Produtos: {len(products_df):,}")
print(f"• Pedidos: {len(orders_df):,}")
print(f"• Itens de pedidos: {len(order_items_df):,}")

print(f"\n📅 PERÍODO DOS DADOS:")
print(f"• Período dos pedidos: {orders_df['order_date'].min()} a {orders_df['order_date'].max()}")
print(f"• Período de registro dos clientes: {customers_df['registration_date'].min()} a {customers_df['registration_date'].max()}")

print(f"\n💰 RECEITA TOTAL: R$ {orders_df['total_amount'].sum():,.2f}")
print(f"📈 TICKET MÉDIO: R$ {orders_df['total_amount'].mean():.2f}")

print(f"\n🛍️ PRODUTOS POR CATEGORIA:")
for category in products_df['category'].unique():
    count = len(products_df[products_df['category'] == category])
    print(f"• {category}: {count} produtos")

print(f"\n📦 STATUS DOS PEDIDOS:")
for status in orders_df['status'].unique():
    count = len(orders_df[orders_df['status'] == status])
    percentage = (count / len(orders_df)) * 100
    print(f"• {status}: {count:,} ({percentage:.1f}%)")

print(f"\n📁 ARQUIVOS CRIADOS:")
print("• customers.csv")
print("• products.csv")
print("• orders.csv")
print("• order_items.csv")
print("• create_tables.sql")

print(f"\n🚀 PRÓXIMOS PASSOS:")
print("1. Execute o script create_tables.sql no seu banco de dados")
print("2. Importe os arquivos CSV para as respectivas tabelas")
print("3. Comece sua análise!")

print("\n" + "="*60)

Mounted at /content/drive
Gerando dados sintéticos para e-commerce...
Gerando clientes...
Gerando produtos...
Gerando pedidos...
Salvando dados em CSV...
Criando scripts SQL...

DADOS GERADOS COM SUCESSO!

📊 ESTATÍSTICAS DOS DADOS:
• Clientes: 10,000
• Produtos: 500
• Pedidos: 50,000
• Itens de pedidos: 101,382

📅 PERÍODO DOS DADOS:
• Período dos pedidos: 2024-01-01 00:00:00 a 2024-12-31 00:00:00
• Período de registro dos clientes: 2023-01-01 a 2024-12-30

💰 RECEITA TOTAL: R$ 60,579,033.13
📈 TICKET MÉDIO: R$ 1211.58

🛍️ PRODUTOS POR CATEGORIA:
• Eletrônicos: 97 produtos
• Esportes: 99 produtos
• Casa & Jardim: 104 produtos
• Livros: 93 produtos
• Moda: 107 produtos

📦 STATUS DOS PEDIDOS:
• Entregue: 42,629 (85.3%)
• Processando: 4,922 (9.8%)
• Cancelado: 2,449 (4.9%)

📁 ARQUIVOS CRIADOS:
• customers.csv
• products.csv
• orders.csv
• order_items.csv
• create_tables.sql

🚀 PRÓXIMOS PASSOS:
1. Execute o script create_tables.sql no seu banco de dados
2. Importe os arquivos CSV para as resp